# Sparse Table

`-` $2$의 거듭제곱 단위로 구간을 분할해 전처리한 정적 구간 쿼리용 자료 구조

## 합성함수와 쿼리

- 문제 출처: [백준 17435번](https://www.acmicpc.net/problem/17435)

`-` 효율적인 최소 공통 조상 구현을 위한 sparse table 구현

`-` 많은 시도 끝에 포기 당했다

`-` 함수 $f: \{1, \dots, m\} \to \{1, \dots, m\}$이므로 비둘기집 원리에 의해 $x$에 $f$를 취하다보면 사이클이 생긴다

`-` 나는 이를 비사이클 부분과 사이클 부분으로 나누어 풀고자 했다

`-` 하지만 실패했고 질문 검색을 보고 희소 배열의 개념을 알게 됐다

`-` $f_n(x)$에서 $n$을 $2$의 거듭제곱꼴로 나누어 연산을 적용한다

`-` 간단하게 $19 = 16 + 2 + 1$이므로 $1$부터 $m$까지의 모든 $x$에 대해 $f_{16}(x), f_2(x), f_1(x)$를 알고 있다면 $f_{19}(x) = f_{16}(f_2(f_1(x)))$로 나타낼 수 있다

`-` 나이브하게 하면 $f$를 $19$번 적용해야 되지만 미리 세팅만 해두면 $3$번만 적용하면 된다

`-` 즉, $O\left(\log n\right)$의 연산이면 $f_n(x)$를 계산할 수 있다

`-` 그럼 이제 $k$는 $2$의 거듭제곱꼴일 때 $f_k(x)$를 미리 계산해보자

`-` 여기서부터는 안 봐서 모른다

`-` 일단 $f_1(x)$ 계산은 쉽고 $f_2(x)$도 마찬가지다

`-` $f_4(1)$을 고려하자

`-` 이건 $f_2(f_2(1))$이다

`-` $f_2(x)$는 알고 있으니 $f_2(f_2(1))$을 계산할 수 있다

`-` 그냥 간단하게 $f_2(x)$를 알고 있고 거기에 $f_2$를 다시 취하면 $f_4$를 계산한 결과이다

`-` 즉, $f_{2k}(x)$를 알기 위해 $f_k(x)$에 $f_k$를 취해주면 된다

`-` 배열의 원소가 $m$개이므로 매 단계마다 $O(m)$의 시간 복잡도를 가진다

`-` 그리고 단계의 횟수는 $O\left(\log n\right)$이므로 배열을 미리 준비하는 것은 $O(m \log n)$의 시간 복잡도를 가진다

`-` 쿼리당 $O(\log n)$의 시간 복잡도를 가지고 쿼리가 $Q$개이므로 $O\left(Q \log n\right)$이다

`-` 따라서 전체 알고리즘으 시간 복잡도는 $O\left(\left(m+Q\right)\log n\right)$이다

In [1]:
import math


def query(n, x, dp):
    binary = bin(n)[2:]
    for i, b in enumerate(binary[::-1]):
        if b == "0":
            continue
        x = dp[i][x]
    return x


def sparse_table(f_results, max_n):
    m = len(f_results)
    dp = [[0] * (m + 1) for _ in range(int(math.log2(max_n)) + 1)]  # dp[i][x]는 f_{2^i}(x)
    for x, f_x in enumerate(f_results, start=1):
        dp[0][x] = f_x
    for i in range(1, int(math.log2(max_n)) + 1):
        for x in range(1, m + 1):
            dp[i][x] = dp[i - 1][dp[i - 1][x]]
    return dp


def solution():
    m = int(input())
    f_results = list(map(int, input().split()))
    max_n = 500000
    dp = sparse_table(f_results, max_n)
    Q = int(input())
    for _ in range(Q):
        n, x = map(int, input().split())
        answer = query(n, x, dp)
        print(answer)


solution()

# input
# 5
# 3 3 5 4 3
# 5
# 1 1
# 2 1
# 11 3
# 1000 4
# 5 1

 5
 3 3 5 4 3
 5
 1 1


3


 2 1


5


 11 3


5


 1000 4


4


 5 1


3


## 개미

- 문제 출처: [백준 14942번](https://www.acmicpc.net/problem/14942)

`-` 희소 배열을 이용해 풀 수 있을 것 같다

`-` 트리의 높이는 최대 $n-1$이며 노드의 개수 $n$은 최대 $10^5$이다

`-` $\log_2\left(10^5\right) \approx 16.6$이므로 임의의 노드에서 루트 노드까지 가는데 $2^{17}$개 미만의 노드를 거친다

`-` 희소 배열을 통해 각 노드에서 $2^i$만큼 떨어진 부모 노드와 거기까지 가는데 소모되는 에너지를 계산하자

`-` 그리고 각 노드에서 $1$번 방을 향해 최대로 거슬러 올라갈 때 도달할 수 있는 방을 계산하면 된다

`-` $i=16$부터 시작해 $i=0$까지 반복하며 에너지가 충분하다면 현재 노드에서 $2^i$만큼 거슬러 올라가면 된다

`-` 전체 알고리즘의 시간 복잡도는 $O\left(n \log n \right)$이다

In [4]:
import math
from collections import defaultdict


def make_tree(tree, node, parent, parents):
    parents[node] = parent
    for child in tree[node]:
        if child == parent:
            continue
        make_tree(tree, child, node, parents)


def make_parent_sparse_table(tree, parents):
    n = len(tree)
    dp = [[NONE] * (LOG_N + 1) for _ in range(n + 1)]  # dp[x][i]는 x의 2^i번째 부모
    for x in range(n + 1):
        dp[x][0] = parents[x]
    for i in range(1, LOG_N + 1):
        for x in range(n + 1):
            dp[x][i] = dp[dp[x][i - 1]][i - 1]
    return dp


def make_weight_sparse_table(tree, parent_sparse_table, weights):
    n = len(tree)
    dp = [[NONE] * (LOG_N + 1) for _ in range(n + 1)]  # dp[x][i]는 x의 2^i번째 부모까지 가는데 필요한 에너지
    for x in range(n + 1):
        p = parent_sparse_table[x][0]
        dp[x][0] = weights[(x, p)]
    for i in range(1, LOG_N + 1):
        for x in range(n + 1):
            p = parent_sparse_table[x][i - 1]
            dp[x][i] = dp[x][i - 1] + dp[p][i - 1]
    return dp


def query(node, energy, parent_sparse_table, weight_sparse_table):
    for i in range(LOG_N, -1, -1):
        w = weight_sparse_table[node][i]
        if energy < w:
            continue
        energy -= w
        node = parent_sparse_table[node][i]
    if node == NONE:
        node = ROOT
    return node


def solution():
    global LOG_N, ROOT, NONE
    n = int(input())
    LOG_N = int(math.log2(n))
    ROOT = 1
    NONE = 0
    ant2energy = [0] * (n + 1)
    for i in range(1, n + 1):
        e = int(input())
        ant2energy[i] = e
    tree = defaultdict(list)
    weights = {}
    for _ in range(n - 1):
        a, b, c = map(int, input().split())
        tree[a].append(b)
        tree[b].append(a)
        weights[(a, b)] = weights[(b, a)] = c
    weights[(NONE, NONE)] = weights[(ROOT, NONE)] = weights[(NONE, ROOT)] = 0
    parents = [NONE] * (n + 1)
    make_tree(tree, ROOT, NONE, parents)
    parent_sparse_table = make_parent_sparse_table(tree, parents)
    weight_sparse_table = make_weight_sparse_table(tree, parent_sparse_table, weights)
    for node in range(1, n + 1):
        answer = query(node, ant2energy[node], parent_sparse_table, weight_sparse_table)
        print(answer)


solution()

# input
# 4
# 10
# 8
# 22
# 18
# 1 2 10
# 2 3 10
# 2 4 10

 4
 10
 8
 22
 18
 1 2 10
 2 3 10
 2 4 10


1
2
1
2
